# Train YOLO Model for Human Detection

In [5]:
import os
import torch
import shutil
from ultralytics import YOLO

In [6]:
# Check if CUDA gpu is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    print("Using CPU")

CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [7]:
# Project file paths
notebook_dir = os.getcwd()  # Current directory (notebooks)
project_root = os.path.dirname(notebook_dir)  # go up one level to get project root
data_dir = os.path.join(project_root, "data")
image_dir = os.path.join(data_dir, "images")
label_dir = os.path.join(data_dir, "labels")
model_dir = os.path.join(project_root, "models")
temp_label_dir = os.path.join(data_dir, "temp_labels")

print(f"Project root: {project_root}")
print(f"Data directory: {data_dir}")
print(f"Model directory: {model_dir}")
print(f"Temporary labels directory: {temp_label_dir}")

Project root: C:\Users\jrom\DataspellProjects\Drowning-Detection
Data directory: C:\Users\jrom\DataspellProjects\Drowning-Detection\data
Model directory: C:\Users\jrom\DataspellProjects\Drowning-Detection\models
Temporary labels directory: C:\Users\jrom\DataspellProjects\Drowning-Detection\data\temp_labels


## 1. Prepare data for YOLO training

In [8]:
# Create temporary labels directory for unified class
def create_temp_labels():
    # Create temp directories
    os.makedirs(os.path.join(temp_label_dir, "train"), exist_ok=True)
    os.makedirs(os.path.join(temp_label_dir, "val"), exist_ok=True)

    # Process train labels
    train_labels = os.path.join(label_dir, "train")
    for label_file in os.listdir(train_labels):
        if label_file.endswith(".txt"):
            input_path = os.path.join(train_labels, label_file)
            output_path = os.path.join(temp_label_dir, "train", label_file)

            with open(input_path, 'r') as infile:
                lines = infile.readlines()

            with open(output_path, 'w') as outfile:
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        # Change class id to 0, keep other values the same
                        new_line = f"0 {' '.join(parts[1:])}\n"
                        outfile.write(new_line)

    # Process validation labels
    val_labels = os.path.join(label_dir, "val")
    for label_file in os.listdir(val_labels):
        if label_file.endswith(".txt"):
            input_path = os.path.join(val_labels, label_file)
            output_path = os.path.join(temp_label_dir, "val", label_file)

            with open(input_path, 'r') as infile:
                lines = infile.readlines()

            with open(output_path, 'w') as outfile:
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        # Change class id to 0, keep other values the same
                        new_line = f"0 {' '.join(parts[1:])}\n"
                        outfile.write(new_line)

    print(f"Created temporary labels with unified class at {temp_label_dir}")

# Temporarily swap label folders
def swap_labels_for_training():
    backup_labels_dir = os.path.join(data_dir, "labels_backup")
    if os.path.exists(backup_labels_dir):
        shutil.rmtree(backup_labels_dir)
    shutil.move(label_dir, backup_labels_dir)
    shutil.copytree(temp_label_dir, label_dir)
    print("Swapped original labels with temporary labels for training.")
    return backup_labels_dir

# Restore original label folder after training
def restore_labels(backup_labels_dir):
    if os.path.exists(label_dir):
        shutil.rmtree(label_dir)
    shutil.move(backup_labels_dir, label_dir)
    print("Restored original labels after training.")

def create_data_yaml():
    yaml_content = f"""
path: {data_dir}
train: images/train
val: images/val
names:
  0: human
"""
    yaml_path = os.path.join(data_dir, "human_detection_data.yaml")
    with open(yaml_path, "w") as f:
        f.write(yaml_content.strip())
    print(f"Created data configuration at {yaml_path}")
    return yaml_path

# Create temporary labels and the data.yaml file
create_temp_labels()
data_yaml = create_data_yaml()
backup_labels_path = swap_labels_for_training()  # Swap labels before training

Created temporary labels with unified class at C:\Users\jrom\DataspellProjects\Drowning-Detection\data\temp_labels
Created data configuration at C:\Users\jrom\DataspellProjects\Drowning-Detection\data\human_detection_data.yaml
Swapped original labels with temporary labels for training.


## 2. Train YOLO11 Model

In [9]:
def train_yolo_model(data_yaml, epochs=50, imgsz=640, batch_size=16):
    print("Loading YOLO11s model...")
    model = YOLO("yolo11s.pt")

    print(f"Training model for {epochs} epochs...")
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch_size,
        device=0 if torch.cuda.is_available() else 'cpu',
        project=model_dir,
        name="human_detection",
        exist_ok=True,
        # Data augmentation settings
        hsv_v=0.4,     # Value (brightness) augmentation
        degrees=10.0,  # Rotation augmentation
        fliplr=0.5,    # Horizontal flip with 50% probability
        scale=0.5,     # Random scaling
        translate=0.1  # Translation augmentation
    )

    # Save the trained model
    model_path = os.path.join(model_dir, "human_detection_yolo11s.pt")
    model.save(model_path)
    print(f"Model saved to {model_path}")

    return model, results

## 3. Validate Model

In [10]:
def validate_model(data_yaml, batch_size=4):
    print("Loading best model for validation...")
    model = YOLO(os.path.join(model_dir, "human_detection", "weights", "best.pt"))  # Load best model

    print("Validating model...")
    results = model.val(
        data=data_yaml,
        batch=batch_size,
        device=0 if torch.cuda.is_available() else 'cpu'
    )

    # Print metrics
    metrics = results.box
    print("\nValidation Results:")
    print(f"mAP50: {metrics.map50.mean():.4f}")
    print(f"mAP50-95: {metrics.map.mean():.4f}")
    print(f"Precision: {metrics.p.mean():.4f}")
    print(f"Recall: {metrics.r.mean():.4f}")

    return results

## 4. Run all steps

In [11]:
model, training_results = train_yolo_model(data_yaml, epochs=20)

Loading YOLO11s model...


100%|██████████| 18.4M/18.4M [00:04<00:00, 4.76MB/s]


Training model for 20 epochs...
New https://pypi.org/project/ultralytics/8.3.111 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.79  Python-3.12.3 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolo11s.pt, data=C:\Users\jrom\DataspellProjects\Drowning-Detection\data\human_detection_data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:\Users\jrom\DataspellProjects\Drowning-Detection\models, name=human_detection, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, s

100%|██████████| 5.35M/5.35M [00:00<00:00, 20.2MB/s]


AMP: checks passed 


...

      20/20      4.05G     0.9912     0.4102      0.996         13        640: 100%|██████████| 438/438 [01:41<00:00,  4.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:13<00:00,  3.83it/s]

                   all       1572       2317      0.994      0.997      0.994      0.777



20 epochs completed in 0.666 hours.
Optimizer stripped from C:\Users\jrom\DataspellProjects\Drowning-Detection\models\human_detection\weights\last.pt, 19.2MB
Optimizer stripped from C:\Users\jrom\DataspellProjects\Drowning-Detection\models\human_detection\weights\best.pt, 19.2MB

Validating C:\Users\jrom\DataspellProjects\Drowning-Detection\models\human_detection\weights\best.pt...
Ultralytics 8.3.79  Python-3.12.3 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:13<00:00,  3.67it/s]


                   all       1572       2317      0.991      0.997      0.994      0.778
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to C:\Users\jrom\DataspellProjects\Drowning-Detection\models\human_detection
Model saved to C:\Users\jrom\DataspellProjects\Drowning-Detection\models\human_detection_yolo11s.pt


In [ ]:
validation_results = validate_model(data_yaml)